In [1]:
import pandas as pd # for manipulating data frames
import pingouin as pg # for running statistics
import plotly.express as px

In [5]:
data = pd.read_csv('logs/logs_merged.csv', sep=',')
data

,DesignName,ParticipantID,TrialID,Block1,Block2,VV,OC,visualSearchTime,ErrorCount
0,Preattentive Variables,1,1,1,1,Shadow,Medium,1552,0
1,Preattentive Variables,1,2,1,2,Shadow,Low,1134,0
2,Preattentive Variables,1,3,1,3,Shadow,High,1140,0
3,Preattentive Variables,1,4,1,4,Shadow,Low,768,0
4,Preattentive Variables,1,5,1,5,Shadow,Medium,904,0
...,...,...,...,...,...,...,...,...,...
265,Preattentive Variables,6,266,3,11,Shadow,Low,858,0
266,Preattentive Variables,6,267,3,12,Shadow,Medium,743,0
267,Preattentive Variables,6,268,3,13,Shadow,Low,648,0
268,Preattentive Variables,6,269,3,14,Shadow,High,676,0


In [6]:
data.iloc[2]

DesignName          Preattentive Variables
ParticipantID                            1
TrialID                                  3
Block1                                   1
Block2                                   3
VV                                  Shadow
OC                                    High
visualSearchTime                      1140
ErrorCount                               0
Name: 2, dtype: object

In [7]:
data.dtypes

DesignName          object
ParticipantID        int64
TrialID              int64
Block1               int64
Block2               int64
VV                  object
OC                  object
visualSearchTime     int64
ErrorCount           int64
dtype: object

In [9]:
data['ParticipantID'] = data['ParticipantID'].astype('str')
data.dtypes

DesignName          object
ParticipantID       object
TrialID              int64
Block1               int64
Block2               int64
VV                  object
OC                  object
visualSearchTime     int64
ErrorCount           int64
dtype: object

In [11]:
### Applying them to a column gives a result of type series
data.visualSearchTime.mean()

3073.0481481481484

In [13]:
### We can get a breakdown by condition using the groupby function
data.groupby('VV').visualSearchTime.mean()

VV
Both      6437.455556
Motion    1511.677778
Shadow    1270.011111
Name: visualSearchTime, dtype: float64

In [14]:
### Applying those aggregating functions to a dataframe gives a result of type dataframe
data.groupby('VV').mean() # result is a dataframe

/tmp/ipykernel_52115/917059096.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data.groupby('VV').mean() # result is a dataframe


,TrialID,Block1,Block2,visualSearchTime,ErrorCount
VV,,,,,
Both,135.5,2.0,8.0,6437.455556,0.122222
Motion,135.5,2.0,8.0,1511.677778,0.011111
Shadow,135.5,2.0,8.0,1270.011111,0.022222


In [15]:
### aggregate is a more elaborate aggregate function
# the line below is equivalent to: data.groupby('Lens').mean()
# data.groupby('Lens').aggregate('mean') 
# but here aggregate is used to specify how to aggregate different columns
data.groupby('VV').aggregate({'TrialID': 'sum', 'visualSearchTime': 'mean'})

,TrialID,visualSearchTime
VV,,
Both,12195,6437.455556
Motion,12195,1511.677778
Shadow,12195,1270.011111
